In [1]:
!pip install torch torchvision

In [2]:
import os
import torch, cv2, pandas as pd, numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import ToTensor, Compose, Resize, Normalize
import matplotlib.pyplot as plt
import torch.nn as nn
root = '/kaggle/input/skin-diseases-image-dataset/IMG_CLASSES'
class buildData(Dataset):
    def __init__(self, root, train=True, transform=None):
        self.transform = transform
        self.root = root
        self.images = []
        self.labels = []
        self.labels_ = []
        for i, folder in enumerate(os.listdir(self.root)):
            path = os.path.join(self.root, folder)
            for imgpath in os.listdir(path):
                self.labels.append(i)
                self.labels_.append(i)
                img = os.path.join(path, imgpath)
                self.images.append(img)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        img = self.images[idx]
        lb = self.labels[idx]
        img = cv2.imread(img)
        img = cv2.resize(img, (224, 224))
        # img = np.transpose(img, (2, 0, 1))
        if self.transform:
            img = self.transform(img)
        return img, lb
transform = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])
data = buildData(root=root, train=True, transform=transform)
img, lb = data.__getitem__(100)
print(img.shape)
train_size = int(0.8 *  data.__len__())
test_size = data.__len__() - train_size
train_dataset, test_dataset = random_split(data, [train_size, test_size])
trainloader = DataLoader(
    train_dataset, batch_size=256, 
    shuffle=True,
    num_workers=16,
)
testloader = DataLoader(
    test_dataset, batch_size=256, 
    shuffle=True,
    num_workers=16,
)

from torchvision import models
num_classes = len(np.unique(data.labels))
print(num_classes)
model = models.resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, num_classes)  
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
epochs=100
criterion = nn.CrossEntropyLoss()
opt = torch.optim.SGD(
    model.parameters(),
    lr=0.001,
    momentum=0.9
)
for epoch in range(epochs):
    model.train()
    for iter, (imgs, lbs) in enumerate(trainloader):
        imgs = imgs.to(device).float()
        lbs = lbs.to(device).long()
        opt.zero_grad()
        pred = model(imgs)
        # print(imgs.shape)
        loss = criterion(pred, lbs)
        loss.backward()
        opt.step()
        if iter % 10 == 0:
            print(f"Epoch {epoch}/{epochs} | Iter {iter}/{len(trainloader)} | Loss: {loss.item()}")

            
    

torch.Size([3, 224, 224])
10


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResN

Epoch 0/100 | Iter 0/85 | Loss: 2.514665126800537
Epoch 0/100 | Iter 10/85 | Loss: 2.0107102394104004
Epoch 0/100 | Iter 20/85 | Loss: 1.6618109941482544
Epoch 0/100 | Iter 30/85 | Loss: 1.4729968309402466
Epoch 0/100 | Iter 40/85 | Loss: 1.3297702074050903
Epoch 0/100 | Iter 50/85 | Loss: 1.1570144891738892
Epoch 0/100 | Iter 60/85 | Loss: 1.2008512020111084
Epoch 0/100 | Iter 70/85 | Loss: 1.1349002122879028
Epoch 0/100 | Iter 80/85 | Loss: 0.9743064045906067
Epoch 1/100 | Iter 0/85 | Loss: 0.9508883357048035
Epoch 1/100 | Iter 10/85 | Loss: 0.9777847528457642
Epoch 1/100 | Iter 20/85 | Loss: 0.8832581043243408
Epoch 1/100 | Iter 30/85 | Loss: 0.9492736458778381
Epoch 1/100 | Iter 40/85 | Loss: 0.9180856943130493
Epoch 1/100 | Iter 50/85 | Loss: 0.8538099527359009
Epoch 1/100 | Iter 60/85 | Loss: 0.8346667289733887
Epoch 1/100 | Iter 70/85 | Loss: 0.8058339953422546
Epoch 1/100 | Iter 80/85 | Loss: 0.8436881303787231
Epoch 2/100 | Iter 0/85 | Loss: 0.7653083205223083
Epoch 2/100 | It

KeyboardInterrupt: 

In [4]:
torch.save(model.state_dict(), 'model.pt')

In [3]:
model = torch.load_state_dict('model.pt')
model.eval()
predictions = []
LabelsVal = []
for epoch in range(epochs):
      for iter, (imgs, lbs) in enumerate(testLoader):
        LabelsVal.extend(lbs)
        with torch.no_grad():
            preds = model(imgs)
            _, id = torch.max(preds, dim=1)
            predictions.extend(id)
            lossVal = criterion(preds, lbs)
predictions = [x.item() for x in predictions]
LabelsVal = [x.item() for x in LabelsVal]
from sklearn.metrics import classification_report

print(classification_report(LabelsVal, predictions))

NameError: name 'testLoader' is not defined